## How convert MegEngine model file to Tengine model file

这里补充 Tengine 介绍和 MegEngine 介绍。
本节教程说明了如何将 MegEngine model 转换为 Tengine model 的过程和注意事项。

### load a MegEngine model form MegEngine.Hub
MegEngine ModHub 包含了采用MegEngine实现的各种主流深度学习模型，提供了各种经典的图像分类、目标检测、图像分割以及自然语言模型的官方实现，每个模型同时提供了模型定义、推理以及训练的代码。
以经典的 ResNet 网络结构为例，MegEngine ModHub 提供了 ImageNet 数据集上的预训练模型和完整训练及测试代码。下面以 resnet18 为例，说明了如何将 MegEngine resnet18 模型转换为 Tengine resnet18 模型。

In [1]:
import megengine.hub

# 只加载网络结构
#resnet18 = megengine.hub.load("megengine/models", "resnet18")

# 加载网络结构和预训练权重
resnet18 = megengine.hub.load("megengine/models", "resnet18", pretrained=True)

04 16:29:21 load_serialized_obj_from_url: download to or using cached /home/i/.cache/megengine/serialized/009607_resnet18_naiveaug_70312_78a63ca6.pkl


对于 MegEngine ModHub 下载的模型和训练得到的原始 MegEngine 模型，需要注意的是模型并未完成*序列化*，需要先将模型序列化后才可进行转换。
详见：https://megengine.org.cn/doc/latest/advanced/deployment.html#id2

In [2]:
from megengine.jit import trace
import megengine.functional as F

# 使用 trace 装饰该函数，详情见 MegEngine 文档“动态图与静态图”、“静态图的两种模式”章节
# pred_fun 经过装饰之后已经变成了 trace 类的一个实例，而不仅仅是一个函数
@trace(symbolic=True)
def pred_fun(data, *, net):
    net.eval()
    pred = net(data)
    # if model has softmax
    pred_normalized = F.softmax(pred)
    return pred_normalized

通过 numpy 声明随机产生一个 (1, 3, 224, 224) 大小的浮点进行输入填充，以便 MegEngine 进行序列化。
需要注意的是，该大小需要和最后部署场景中的大小有关，具体取决于网络的结构限制，比如 FC Layer。

In [3]:
import numpy as np
data = np.random.random((1, 3, 224, 224)).astype(np.float32)

In [4]:
pred_fun.trace(data, net=resnet18)
pred_fun.dump('new_model.pkl')

将被 trace 装饰的函数（这里的 pred_fun ）视为计算图的全部流程，计算图的输入严格等于 pred_fun 的位置参数（positional arguments，即参数列表中星号 * 前的部分，这里的 data 变量），计算图的输出严格等于函数的返回值（这里的 pred_normalized ）。而这也会进一步影响到部署时模型的输入和输出，即如果运行部署后的该模型，会需要一个 data 格式的输入，返回一个 pred_normalized 格式的值。
为了便于我们在 C++ 代码中给序列化之后的模型传入输入数据，我们需要给输入赋予一个名字，即代码中的 arg_names 参数。由于该示例中 pred_fun 只有一个位置参数，即计算图只有一个输入，所以传给 arg_names 的列表也只需一个字符串值即可，可以是任意名字，用于在 C++ 代码中引用，详情见下节内容。另外传递 optimize_for_inference=True （默认为 Flase）标志将在 dump() 方法中对模型针对 inference 进行优化，提高 inference 时候的模型运行效率。

最后输出的 new_model.pkl 就是序列化后的 MegEngine 模型了。

通过下面的命令，可以将 MegEngine 模型转换为 Tengine tmfile 格式的模型：
``` bash
convert_tool -f megengine -m new_model.pkl -o res18_megengine.tmfile
```